In [1]:
import warnings
warnings.simplefilter("ignore")
import gzip
import json
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import geopandas as gpd

from collections import Counter


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Example connection string (modify based on your database)
DB_HOST = "localhost"  # e.g., "localhost"
DB_PORT = "5432"  # Default PostgreSQL port
DB_NAME = "Capstone"
DB_USER = "postgres"
DB_PASSWORD = "Google#13"
TABLE_NAME = "main_new"
CHUNK_SIZE = 600  # Adjust as needed

# Create a SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# SQL query to fetch user_id and text
query = "SELECT user_id, id as item_id, rating, time as timestamp  FROM public.main_new where category like '%%restaurant%%' ;"

# Read into DataFrame
df = pd.read_sql(query, engine)



In [5]:
df.to_csv("Modified_Reviews_Metadta.csv")

In [13]:
df['user_id'] = df['user_id'].astype(str)
df['item_id'] = df['item_id'].astype(str)


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6133627 entries, 0 to 6133626
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    object        
 1   item_id    object        
 2   rating     float64       
 3   timestamp  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 187.2+ MB


In [ ]:
# To use the Lenskit algorithm and fraction of 0.2 removed users with reviewed items less than 20

filtered_df = df.groupby(["user_id"]).filter(lambda x: len(x) >= 20)

In [21]:
filtered_df.head()

,user_id,item_id,rating,timestamp
44,1.1336297284308268e+20,48,4.0,2004-02-19
45,1.1336297284308268e+20,49,5.0,2004-02-19
46,1.1336297284308268e+20,50,4.0,2004-02-19
97,1.1257611969463946e+20,123,4.0,2006-02-17
111,1.1257611969463946e+20,126,4.0,2006-02-17


<h2> Lenskit Implementation</h2>

In [ ]:
# Import all the libraries
from lenskit.als import BiasedMFScorer
from lenskit.batch import recommend
from lenskit.data import ItemListCollection, UserIDKey, from_interactions_df
from lenskit.knn import ItemKNNScorer
from lenskit.metrics import NDCG, RBP, RecipRank, RunAnalysis

from lenskit.splitting import SampleFrac, crossfold_users
from lenskit.pipeline import topn_pipeline

In [ ]:
# Convet the dataset into lenskit supportable format

data = from_interactions_df(filtered_df)

In [ ]:
model_als = BiasedMFScorer(features=50)
pipe_als = topn_pipeline(model_als)

In [ ]:
# test data is organized by user
all_test = ItemListCollection(UserIDKey)
# recommendations will be organized by model and user ID
all_recs = ItemListCollection(["model", "user_id"])


In [ ]:
for split in crossfold_users(data, 5, SampleFrac(0.2)):
    # collect the test data
    all_test.add_from(split.test)

    # train the pipeline, cloning first so a fresh pipeline for each split
    fit_als = pipe_als.clone()
    fit_als.train(split.train)
   
     # generate recs
    als_recs = recommend(fit_als, split.test.keys(), 50)
    all_recs.add_from(als_recs, model="ALS")


In [ ]:
all_test.to_df().to_csv("test_data.csv")

In [ ]:
result = all_recs.to_df()

In [ ]:
result.to_csv("results.csv")